In [20]:
import json
import numbers
import os
from string import Template
import tempfile

from IPython.display import IFrame

content = Template('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta content="text/html;charset=utf-8" http-equiv="Content-Type">
    <meta content="utf-8" http-equiv="encoding">
</head>
<title>Parallel Coordinates Graduation Stephanie Case Study 1</title>

<link rel="stylesheet" type="text/css" href="css/d3.parcoords.css">
<link rel="stylesheet" type="text/css" href="css/styling.css">
<script src="js/d3.js"></script>
<script src="js/d3.svg.multibrush.js"></script>
<script src="js/sylvester.js"></script>
<script src="js/underscore.js"></script>
<script src="js/slickgrid/jquery-1.7.min.js"></script>
<script src="js/underscore.js"></script>
<script src="js/underscore.math.js"></script>
<script src="js/d3.parcoords.js"></script>
<script src="http://d3js.org/queue.v1.min.js"></script>
<script src="js/canvas2svg.js"></script>
<script src="js/FileSaver.js"></script>
<script src="js/vkbeautify.0.99.00.beta.js"></script>

<body>
<div id="example" class="parcoords" style="width:2600px;height:1000px""></div>
</body>

<script>

// load csv file and create the chart
var data = $data

// color scale for zscores
var colorscale = d3.scale.linear()
    .domain([0,1])
    .range(["blue", "red"])
    .interpolate(d3.interpolateLab);

var pc0 = d3.parcoords()("#example")
    .data(data)
    .bundlingStrength(0) // set bundling strength
    .smoothness(0.0)
    .showControlPoints(false)
    .hideAxis(["name"])
    .composite("darken")
    .render()
    .brushMode("1D-axes")
    .reorderable()  // enable brushing
    .interactive();

const axes = pc0.svg.selectAll(".axis")[0];

const textlabels = pc0.svg.selectAll('.resize')
    .append("text")
    .attr("dy", function(n, i){
            if (n=='s'){
                return "+15";
            }
            if (n=='n'){
                return "-5";
            }});


console.log(textlabels)

textlabels
    .attr({
          "text-anchor": "middle", 
          "font-weight": "bold"
            });


pc0.svg.selectAll("text")
    .style("font", "10px");


// click label to activate coloring
pc0.svg.selectAll(".dimension")
    .on("click", change_color)
    .selectAll(".label")
    .style("font-size", "16px");

    pc0.on("brush", function(d){
       
        let lower, upper, index, label;

        textlabels
            .text(
                function(n,i){
                // n is upper (s) or lower (n)
                // i is index of axis

                index = Math.floor(i/2);
                label = axes[index].__data__;

                    if (n=='s'){
                        // y depends on i
                        lower = Math.min.apply(Math,d.map(function(o){return o[label];}));
                        return lower.toFixed(0)
                        ;
                    }
                    if (n=='n'){
                        // y depends on i
                        //Math.max.apply(Math,array.map(function(o){return o[y];}))
                        upper = Math.max.apply(Math,d.map(function(o){return o[label];}));

                        return upper.toFixed(0)
                        ;

                    }
                }

            )
            // .attr("dy", function(n, i){
            //     if (n=='s'){
            //         return "+15";
            //     }
            //     if (n=='n'){
            //         return "0";
            //     }})

    });
    // elke text kan achterhalen via function(d, n),
    // om welke dimensie het om gaat
    // maar dan ben je er nog niet, want in je
    // brush event weet je niet welke as geselecteerd is
    // tenzij je gewoon alle labels update
    // maar hoe kan je alle labels updaten? door voor
    // elk text item vast te leggen welke dimensie het is
    //
    // andere optie is d3.parcoords zelf aanpassen en as waarop
    // gebrushed wordt mee te geven

function saveSVG(){
}

// helper function for saving svg
function setAttributeByTag(xmlDocument, tagName, attribute, value){
    const paths = xmlDocument.getElementsByTagName(tagName);
    for (let i = 0; i < paths.length; i++) {
        paths[i].setAttribute(attribute, value);
    }
}

// helper function for saving svg
function setAttributeByClass(xmlDocument, className, attribute, value){
    const paths = xmlDocument.getElementsByClassName(className);
    for (let i = 0; i < paths.length; i++) {
        paths[i].setAttribute(attribute, value);
    }
}




// update color
function change_color(dimension) {
    pc0.svg.selectAll(".dimension")
        .style("font-weight", "normal")
        .filter(function(d) { return d == dimension; })
        .style("font-weight", "normal")

    pc0.color(color(pc0.data(),dimension)).render()
} 	

// return color function based on plot and dimension
// function color(col, dimension) {
 //   var normed = normalize(_(col).pluck(dimension).map(parseFloat))
   // return function(d) { return colorscale(normed(d[dimension])) }
//};

// normalize
function normalize(col) {
    var n = col.length,
        min = _(col).min()
    max = _(col).max()

    return function(d) {
        return (d-min)/(max-min);
    };
}

</script>

<button onclick="saveSVG();">Save as SVG File</button>

</html>
''')

def data_to_string(data):
    '''format a pandas data frame in a string of dicts, with keys in the order
    they appear in the dataframe
    
    Parameters
    ----------
    data  : pd.Dataframe instance
    
    
    Returns
    -------
    str
    
    '''
    
    dict_data = data.to_dict(orient='records')
    ordered_keys = data.columns.values
    
    string_elements = []
    
    for entry in dict_data:
        elements = []
        for key in ordered_keys:
            value = entry[key]
            elements.append("'{}': {}".format(key, value))
        elements = ', '.join(elements)
        string_elements.append('{' + elements +'}')
    return "[" + ', '.join(string_elements) + "]"

class generate_html(object):
    '''
    
    Parameters
    ----------
    x : pandas dataframe
    fn : str, optional
         file name, if not provided a temporary file is used
    
    '''
    
    # we need to keep a record of all the tempfiles, otherwise the
    # tempfiles are removed after the call, resulting in a 404 error
    tempfiles = []
    
    def __call__(self, x, fn=None):
        if fn is None:
            fh = tempfile.NamedTemporaryFile(suffix='.html', 
                        dir='.',
                        mode='w+t')
        else:
            fh = open(fn, 'w')
        
        data = data_to_string(x)

        formatted = content.substitute({'data':data})
        fh.write(formatted)
        fh.seek(0)
    
        if fn is None:
            self.tempfiles.append(fh)
        
        file_loc = './{}'
        file_path = file_loc.format(os.path.basename(fh.name))
        
        return IFrame(file_path, 1000,800)

# instantiate the class, and make it look as if it is a simple function
generate_html = generate_html()

In [21]:
from ema_workbench import load_results
results = load_results('./data/all policies 50 experiments.tar.gz')

In [22]:
experiments, outcomes = results
x = pd.DataFrame.from_records(experiments)
y = outcomes

In [23]:
# generate_html(x, y)
x.drop('model', axis=1, inplace=True)
x.head()

,DamFunctTbl_multiplier,ShipTbl1,ShipTbl2,ShipTbl3,climate scenarios,collaboration,fragility dikes,land use scenarios,policy
0,-0.071856,-0.090380,0.044701,0.088807,24,1.185791,0.066317,sustainableGrowth,RfR Small Scale
1,0.079882,0.096824,-0.016479,0.079280,1,1.338038,-0.012839,urbanizationLargeSteady,RfR Small Scale
2,-0.032444,0.055302,0.049098,0.084686,15,1.230366,0.006924,moreNature,RfR Small Scale
3,-0.048025,0.089139,-0.072499,0.032402,7,1.002401,0.085070,moreNature,RfR Small Scale
4,0.047745,-0.038221,0.058771,-0.070511,21,1.455446,0.093555,Deurbanization,RfR Small Scale


In [24]:
x['land use scenarios'] = pd.Categorical(x['land use scenarios'] )
x.policy = pd.Categorical(x.policy )

In [25]:
temp = {}
y = x.copy()
for key, value in outcomes.items():
    if value.ndim==2:
        value = np.sum(value, axis=1)
    y = y.assign(**{key:value})

In [26]:
y.policy = y.policy.cat.codes
y['land use scenarios'] = y['land use scenarios'].cat.codes

In [27]:
y.head()

,DamFunctTbl_multiplier,ShipTbl1,ShipTbl2,ShipTbl3,climate scenarios,collaboration,fragility dikes,land use scenarios,policy,Number of casualties,Flood damage (Milj. Euro),Costs
0,-0.071856,-0.090380,0.044701,0.088807,24,1.185791,0.066317,3,19,686.37921,42196.0,70.0
1,0.079882,0.096824,-0.016479,0.079280,1,1.338038,-0.012839,6,19,486.34140,11525.0,70.0
2,-0.032444,0.055302,0.049098,0.084686,15,1.230366,0.006924,2,19,377.86570,12749.0,70.0
3,-0.048025,0.089139,-0.072499,0.032402,7,1.002401,0.085070,2,19,529.13143,12969.0,70.0
4,0.047745,-0.038221,0.058771,-0.070511,21,1.455446,0.093555,0,19,528.60296,22524.0,70.0


In [29]:
generate_html(y, 'test.html')